# waves_by_cowclip

Notebook environment to migrate csv files to CF compliant zarr

In [1]:
# Optional; code formatter, installed as jupyter lab extension
#%load_ext lab_black
# Optional; code formatter, installed as jupyter notebook extension
%load_ext nb_black

<IPython.core.display.Javascript object>

### Configure OS independent paths

In [332]:
# Import standard packages
import os
import pathlib

import sys
import numpy as np
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import xarray as xr
import math
import itertools
import glob
from copy import deepcopy
from datacube.utils.cog import write_cog

# Import custom functionality
from coclicodata.drive_config import p_drive
from coclicodata.etl.cf_compliancy_checker import check_compliancy, save_compliancy

# Define (local and) remote drives
dir_path_data = p_drive.joinpath('11209199-climate-resilient-ports','00_general','02_COWCLIP_Harvest_Morim_et_al','output_NetCDF')

# Workaround to the Windows OS (10) udunits error after installation of cfchecker: https://github.com/SciTools/iris/issues/404
os.environ["UDUNITS2_XML_PATH"] = str(
    pathlib.Path().home().joinpath(  # change to the udunits2.xml file dir in your Python installation
        r"AppData\Local\mambaforge\pkgs\udunits2-2.2.28-h892ecd3_0\Library\share\udunits\udunits2.xml"
    )
)

# Project paths & files (manual input)
dir_path_cf = dir_path_data.joinpath(r"CF")  # directory to save output CF check files
dir_path_cog = dir_path_data.joinpath(r"COG")  # directory to save output COG files
file_path_cf_merged_nc = dir_path_data.joinpath('waves_by_cowclip_cf_merged.nc')  # file to save merged netCDF file

<IPython.core.display.Javascript object>

In [3]:
dir_path_data

WindowsPath('P:/11209199-climate-resilient-ports/00_general/02_COWCLIP_Harvest_Morim_et_al/output_NetCDF')

<IPython.core.display.Javascript object>

### Get NC-files

In [4]:
# Get all nc files in directory and subdirectories if does not contain 'cf' in filename
file_path_ls = glob.glob(str(dir_path_data.joinpath("**/*.nc")), recursive=True)
file_path_ls = [f for f in file_path_ls if "cf" not in f]

<IPython.core.display.Javascript object>

In [5]:
file_path_ls

['P:\\11209199-climate-resilient-ports\\00_general\\02_COWCLIP_Harvest_Morim_et_al\\output_NetCDF\\Dm_avg\\median_Dm_avg_under_rcp45.nc',
 'P:\\11209199-climate-resilient-ports\\00_general\\02_COWCLIP_Harvest_Morim_et_al\\output_NetCDF\\Dm_avg\\median_Dm_avg_under_rcp85.nc',
 'P:\\11209199-climate-resilient-ports\\00_general\\02_COWCLIP_Harvest_Morim_et_al\\output_NetCDF\\Dm_avg\\percentiles\\p000_Dm_avg_under_rcp45.nc',
 'P:\\11209199-climate-resilient-ports\\00_general\\02_COWCLIP_Harvest_Morim_et_al\\output_NetCDF\\Dm_avg\\percentiles\\p000_Dm_avg_under_rcp85.nc',
 'P:\\11209199-climate-resilient-ports\\00_general\\02_COWCLIP_Harvest_Morim_et_al\\output_NetCDF\\Dm_avg\\percentiles\\p005_Dm_avg_under_rcp45.nc',
 'P:\\11209199-climate-resilient-ports\\00_general\\02_COWCLIP_Harvest_Morim_et_al\\output_NetCDF\\Dm_avg\\percentiles\\p005_Dm_avg_under_rcp85.nc',
 'P:\\11209199-climate-resilient-ports\\00_general\\02_COWCLIP_Harvest_Morim_et_al\\output_NetCDF\\Dm_avg\\percentiles\\p095_Dm_

<IPython.core.display.Javascript object>

### Check CF compliancy original NetCDF files

In [6]:
# open first dataset
ds = xr.open_dataset(file_path_ls[0])

# check original dataset
ds

<xarray.Dataset>
Dimensions:     (longitude: 361, latitude: 181)
Dimensions without coordinates: longitude, latitude
Data variables:
    LAT         (longitude, latitude) float32 ...
    LON         (longitude, latitude) float32 ...
    hist_value  (longitude, latitude) float32 ...
    pred_value  (longitude, latitude) float32 ...
    diff_value  (longitude, latitude) float32 ...
Attributes:
    Description:  median_Dm_avg_under_rcp45
    Variable:     Median Average wave direction

<IPython.core.display.Javascript object>

In [7]:
%%capture cap --no-stderr
# check original CF compliancy (for first file)

check_compliancy(testfile=pathlib.Path(file_path_ls[0]), working_dir=dir_path_cf)

<IPython.core.display.Javascript object>

In [8]:
# save original CF compliancy (for first file)
save_compliancy(cap, testfile=pathlib.Path(file_path_ls[0]), working_dir=dir_path_cf)

{'P:\\11209199-climate-resilient-ports\\00_general\\02_COWCLIP_Harvest_Morim_et_al\\output_NetCDF\\Dm_avg\\median_Dm_avg_under_rcp45.nc': {'warnings': '6', 'errors': '0'}}


<IPython.core.display.Javascript object>

### Make CF compliant alterations to the NetCDF files (dataset dependent)

In [220]:
# open datasets
ds_ls = []
for file_path in file_path_ls:
    ds_ls.append(xr.open_dataset(file_path))

<IPython.core.display.Javascript object>

In [221]:
import json
# NetCDF attribute alterations by means of metadata template
f_global = open(dir_path_data.joinpath("metadata_waves_by_cowclip.json"))
meta_global = json.load(f_global)

# Add metadata to datasets
for ds in ds_ls:
    for attr_name, attr_val in meta_global.items():
        if attr_name == 'PROVIDERS':
            attr_val = json.dumps(attr_val)
        ds.attrs[attr_name] = attr_val

    ds.attrs['Conventions'] = "CF-1.8"

<IPython.core.display.Javascript object>

In [222]:
def transpose_dims_of_vars(ds, vars=[], order=[]):
    """ Transpose dimensions of variables in dataset. """
    # Get all variables if none are specified
    if len(vars) == 0:
        vars = list(ds.data_vars)
    
    # Get reversed order of dimensions if none is specified
    if len(order) == 0:
        order = tuple(reversed(tuple(range(len(ds[vars[0]].dims)))))

    # Get dimensions
    dims = list(ds[vars[0]].dims)

    # Transpose dimensions
    dims = [dims[i] for i in order]

    for var in vars:
        # Get values
        vals = ds[var].values

        # Transpose values
        vals = vals.transpose(*order)

        # Add variable to dataset
        if var in dims:
            ds = ds.assign_coords({var: (dims, vals)})
        else:
            ds = ds.assign({var: (dims, vals)})

        
    return ds

def assign_coords_to_dim(ds, dim, coords):
    """ Assign coordinates to dimension. """
    # Assign coordinates to dimension
    ds = ds.assign_coords({dim: coords})
    
    # Convert dimension to string if it is an object
    if type(coords[0]) == str:
        ds[dim] = ds[dim].astype('S')
    
    # Reset index
    #ds = ds.reset_index(dim)

    return ds

def add_dim(ds, dim, coords, axis=-1):
    """ Add dimension to dataset. """
    # Add dimension to dataset
    if axis == None:
        ds = ds.expand_dims(dim={dim: coords})
    ds = ds.expand_dims(dim={dim: coords}, axis=axis)

    # Assign coordinates to dimension
    ds = assign_coords_to_dim(ds, dim, coords)
    
    return ds

def merge_vars(ds, dim, var, vars=[], coords=[], axis=-1):
    """ Merge variables into one variable. """
    # Get all variables if none are specified
    if len(vars) == 0:
        vars = list(ds.data_vars)

    # Coordinates are the same as the variables if none are specified
    if len(coords) == 0:
        coords = vars

    # Get dimensions and values
    dims = list(ds[vars[0]].dims) + [dim]
    vals = ds[vars[0]].values

    # Merge values
    vals = np.stack([ds[var].values for var in vars], axis=-1)

    # Remove variables
    ds = ds.drop_vars(vars)

    # Add dimension to dataset
    ds = add_dim(ds, dim, coords)

    # Add variable to dataset
    ds[var] = xr.DataArray(vals, dims=dims)

    # Reorder dimensions
    if axis != -1:
        dims_r = dims.copy()
        dims_r.pop(-1)
        dims_r.insert(axis, dim)
        ds = ds.transpose(*dims_r)
    
    return ds

# Assign atrributes to variables and coordinates
def assign_attributes(ds, var, attrs):
    """ Assign attributes to variable or coordinate. """
    # Assign attributes to variable or coordinate
    ds[var].attrs = attrs
    
    return ds

<IPython.core.display.Javascript object>

In [12]:
# Check file path
file_path_ls[0]

'P:\\11209199-climate-resilient-ports\\00_general\\02_COWCLIP_Harvest_Morim_et_al\\output_NetCDF\\Dm_avg\\median_Dm_avg_under_rcp45.nc'

<IPython.core.display.Javascript object>

In [223]:
ds['longitude'].values

array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
        78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
        91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
       104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
       117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129,
       130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142,
       143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155,
       156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168,
       169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 18

<IPython.core.display.Javascript object>

In [225]:
((ds['longitude'] + 180) % 361) - 180

<xarray.DataArray 'longitude' (longitude: 361)>
array([   0,    1,    2,    3,    4,    5,    6,    7,    8,    9,   10,
         11,   12,   13,   14,   15,   16,   17,   18,   19,   20,   21,
         22,   23,   24,   25,   26,   27,   28,   29,   30,   31,   32,
         33,   34,   35,   36,   37,   38,   39,   40,   41,   42,   43,
         44,   45,   46,   47,   48,   49,   50,   51,   52,   53,   54,
         55,   56,   57,   58,   59,   60,   61,   62,   63,   64,   65,
         66,   67,   68,   69,   70,   71,   72,   73,   74,   75,   76,
         77,   78,   79,   80,   81,   82,   83,   84,   85,   86,   87,
         88,   89,   90,   91,   92,   93,   94,   95,   96,   97,   98,
         99,  100,  101,  102,  103,  104,  105,  106,  107,  108,  109,
        110,  111,  112,  113,  114,  115,  116,  117,  118,  119,  120,
        121,  122,  123,  124,  125,  126,  127,  128,  129,  130,  131,
        132,  133,  134,  135,  136,  137,  138,  139,  140,  141,  142,
        143,  144,  145,  146,  147,  148,  149,  150,  151,  152,  153,
        154,  155,  156,  157,  158,  159,  160,  161,  162,  163,  164,
        165,  166,  167,  168,  169,  170,  171,  172,  173,  174,  175,
        176,  177,  178,  179,  180, -180, -179, -178, -177, -176, -175,
       -174, -173, -172, -171, -170, -169, -168, -167, -166, -165, -164,
       -163, -162, -161, -160, -159, -158, -157, -156, -155, -154, -153,
       -152, -151, -150, -149, -148, -147, -146, -145, -144, -143, -142,
       -141, -140, -139, -138, -137, -136, -135, -134, -133, -132, -131,
       -130, -129, -128, -127, -126, -125, -124, -123, -122, -121, -120,
       -119, -118, -117, -116, -115, -114, -113, -112, -111, -110, -109,
       -108, -107, -106, -105, -104, -103, -102, -101, -100,  -99,  -98,
        -97,  -96,  -95,  -94,  -93,  -92,  -91,  -90,  -89,  -88,  -87,
        -86,  -85,  -84,  -83,  -82,  -81,  -80,  -79,  -78,  -77,  -76,
        -75,  -74,  -73,  -72,  -71,  -70,  -69,  -68,  -67,  -66,  -65,
        -64,  -63,  -62,  -61,  -60,  -59,  -58,  -57,  -56,  -55,  -54,
        -53,  -52,  -51,  -50,  -49,  -48,  -47,  -46,  -45,  -44,  -43,
        -42,  -41,  -40,  -39,  -38,  -37,  -36,  -35,  -34,  -33,  -32,
        -31,  -30,  -29,  -28,  -27,  -26,  -25,  -24,  -23,  -22,  -21,
        -20,  -19,  -18,  -17,  -16,  -15,  -14,  -13,  -12,  -11,  -10,
         -9,   -8,   -7,   -6,   -5,   -4,   -3,   -2,   -1], dtype=int64)
Dimensions without coordinates: longitude

<IPython.core.display.Javascript object>

In [226]:
# NetCDF variable and dimension alterations (per dataset)
ds_cf_ls = []
for ds, file_path in zip(ds_ls, file_path_ls):
    # Get information from file_path
    file_name = file_path.split('\\')[-1].split('.')[0]
    file_name_split = file_name.split('_')
    ens_perc = file_name_split[0]
    ens_perc = 50 if ens_perc == 'median' else int(ens_perc[1:])
    var = file_name_split[1]
    var_stat = file_name_split[2]
    rcp = float(file_name_split[4][3:5])/10

    # Get information from paper
    periods_str = [['1979-01-01', '2004-01-01'], ['2081-01-01', '2100-01-01']]
    periods = []
    for period_str in periods_str:
        periods.append([pd.to_datetime(period_str[0]), pd.to_datetime(period_str[1])])
    times = [period[0] + (period[1] - period[0])/2 for period in periods] # Mean of periods

    # Rename dimensions and variables
    ds = ds.rename_dims({"latitude": "lat","longitude": "lon"})
    ds = ds.rename_vars({"LAT": "lat", "LON": "lon"})

    # Transpose dimensions of variables (lat, lon)
    ds = transpose_dims_of_vars(ds)

    # Set coordinates
    ds = ds.set_coords(['lat','lon'])

    # Correct longitude values from 0-360 to -180-180
    ds['lon'] = ((ds['lon'] + 180) % 361) - 180

    # Sort data by ascending longitude
    for var_ in ds.data_vars:
        values  = ds[var_].values
        lons = ds['lon'].values
        idx = np.argsort(lons[0])
        values = values[:,idx]
        lons = lons[:,idx]
        ds[var_].values = values
    ds['lon'].values = lons
    
    # Merge variables
    ds = merge_vars(ds, dim='time', var=var, vars=['hist_value','pred_value'], coords=times, axis=0)
    
    # Drop variables
    if 'diff_value' in ds.data_vars:
        ds = ds.drop_vars(['diff_value'])
    if 'diff_perc_value' in ds.data_vars:
        ds = ds.drop_vars(['diff_perc_value'])

    # Get
    lats = ds['lat'].values
    lons = ds['lon'].values
    times = ds['time'].values
    values = ds[var].values
    
    # Flatten values
    lats = lats[:,0]
    lons = lons[0,:]
    #stations = range(len(lats))

    # Flatten last two dimensions of values
    #values = values.reshape((values.shape[0], -1))

    ds2 = xr.Dataset()
    #ds2 = ds2.assign_coords({'stations': stations})
    ds2 = ds2.assign_coords({'lat': lats})
    ds2 = ds2.assign_coords({'lon': lons})
    ds2 = ds2.assign({'time': ('time', times)})
    ds2 = ds2.assign({'rcp': ('rcp', [rcp])})
    ds2 = ds2.assign({'ens_perc': ('ens_perc', [ens_perc])})
    ds2 = ds2.assign({'var_stat': ('nvar_stat', np.array([var_stat], dtype='S'))})

    # Make lon and lat coordinates
    ds2 = ds2.set_coords(['lat','lon'])
    ds2 = ds2.set_coords(['var_stat'])

    # Add values (sations by time)
    ds2 = ds2.assign({var: (['time', 'lat', 'lon', 'rcp', 'ens_perc', 'nvar_stat'], values.reshape((values.shape[0], values.shape[1], values.shape[2], 1, 1, 1)))})

    # Drop coordinate
    #ds2 = ds2.drop(['stations'])

    # Add attributes for coordinates
    ds2 = assign_attributes(ds2, var='lat', attrs={'standard_name': 'latitude', 'long_name': 'latitude', 'units': 'degrees_north'})
    ds2 = assign_attributes(ds2, var='lon', attrs={'standard_name': 'longitude', 'long_name': 'longitude', 'units': 'degrees_east'})
    ds2 = assign_attributes(ds2, var='time', attrs={'standard_name': 'time', 'long_name': 'time', 'units': 'years'})
    ds2 = assign_attributes(ds2, var='ens_perc', attrs={'long_name': 'model ensemble percentile', 'units': '1'})
    ds2 = assign_attributes(ds2, var='var_stat', attrs={'long_name': 'variable statistic', 'units': '1'})
    ds2 = assign_attributes(ds2, var='rcp', attrs={'long_name': 'radiative forcing pathway', 'units': '1'})

    # Add attributes for variables
    variable_attr_dict = {'Hs': {'long_name': 'significant wave height', 'units': 'm'},
                          'Tm': {'long_name': 'wave period', 'units': 's'},
                          'Dm': {'long_name': 'wave direction', 'units': 'degrees'}}
    
    ds2 = assign_attributes(ds2, var=var, attrs=variable_attr_dict[var])
    
    # Remove times unit in order to save as netCDF
    ds2['time'].attrs.pop('units')
    

    # Add to list of datasets
    ds_cf_ls.append(ds2)

print('Number of datasets: {}'.format(len(ds_cf_ls)))

Number of datasets: 160


<IPython.core.display.Javascript object>

In [231]:
# Check dataset
ds_cf_ls[0]

<xarray.Dataset>
Dimensions:   (lat: 181, lon: 361, time: 2, rcp: 1, ens_perc: 1, nvar_stat: 1)
Coordinates:
  * lat       (lat) float32 -90.0 -89.0 -88.0 -87.0 ... 87.0 88.0 89.0 90.0
  * lon       (lon) float32 -180.0 -179.0 -178.0 -177.0 ... 178.0 179.0 180.0
  * time      (time) datetime64[ns] 1991-07-02T12:00:00 2090-07-02T12:00:00
  * rcp       (rcp) float64 4.5
  * ens_perc  (ens_perc) int32 50
    var_stat  (nvar_stat) |S3 b'avg'
Dimensions without coordinates: nvar_stat
Data variables:
    Dm        (time, lat, lon, rcp, ens_perc, nvar_stat) float32 nan nan ... nan

<IPython.core.display.Javascript object>

In [232]:
# Function to split datasets
def split_dataset(ds_ls):
    # Initiate list of datasets
    ds_ls_split = []
    
    # Split datasets
    for ds in ds_ls:
        for var in ds.data_vars:
            # Copy dataset
            ds_copy = deepcopy(ds)

            # Drop variables except for variable of interest
            ds_copy = ds_copy.drop_vars([var_ for var_ in ds_copy.data_vars if var_ != var])

            # Add to list of datasets
            ds_ls_split.append(ds_copy)
    
    # Return list of datasets
    return ds_ls_split

<IPython.core.display.Javascript object>

In [233]:
ds_cf_split_ls = split_dataset(ds_cf_ls)

print('Number of datasets: {}'.format(len(ds_cf_split_ls)))

Number of datasets: 160


<IPython.core.display.Javascript object>

In [234]:
# Pad data to make all dimensions same size
def pad_dataset(ds_ls):
    # Initiate list of datasets
    ds_ls_pad = []

    # Get maximum dimensions
    dims_max = {}
    for ds in ds_ls:
        for dim in ds.dims:
            if dim not in dims_max:
                dims_max[dim] = ds.dims[dim]
            else:
                dims_max[dim] = max(dims_max[dim], ds.dims[dim])
    
    return ds_ls_pad, dims_max

# Pad datasets
ds_ls_pad, dims_max = pad_dataset(ds_cf_split_ls)

<IPython.core.display.Javascript object>

In [235]:
# Check dataset
ds_cf_split_ls[0]

<xarray.Dataset>
Dimensions:   (lat: 181, lon: 361, time: 2, rcp: 1, ens_perc: 1, nvar_stat: 1)
Coordinates:
  * lat       (lat) float32 -90.0 -89.0 -88.0 -87.0 ... 87.0 88.0 89.0 90.0
  * lon       (lon) float32 -180.0 -179.0 -178.0 -177.0 ... 178.0 179.0 180.0
  * time      (time) datetime64[ns] 1991-07-02T12:00:00 2090-07-02T12:00:00
  * rcp       (rcp) float64 4.5
  * ens_perc  (ens_perc) int32 50
    var_stat  (nvar_stat) |S3 b'avg'
Dimensions without coordinates: nvar_stat
Data variables:
    Dm        (time, lat, lon, rcp, ens_perc, nvar_stat) float32 nan nan ... nan

<IPython.core.display.Javascript object>

In [236]:
# Function to get merge dataframe 
def get_merge_dataframe(ds_ls, merge_keys):
    # Create merge list
    merge_ls = []
    for i, ds in enumerate(ds_ls):
        merge_dict = {}
        merge_dict = {'index': i}
        for key in merge_keys:
            if key in ds.coords: # if key is a coordinate
                merge_dict[key] = ds[key].values[0]
            elif key == 'variables': # if key is variables
                merge_dict[key] = list(ds.keys())[0]
            else: # if key is not a coordinate
                merge_dict[key] = ''
        merge_ls.append(merge_dict)
    
    # Create dataframe
    merge_df = pd.DataFrame(merge_ls)

    # Set types of columns
    for key in merge_keys:
        if key in ds.coords: # if key is a coordinate
            merge_df[key] = merge_df[key].astype(ds_ls[0][key].dtype)
        elif key == 'variables': # if key is variables
            merge_df[key] = merge_df[key].astype(str)
        else: # if key is not a coordinate
            merge_df[key] = merge_df[key].astype(str)

    # Sort dataframe
    merge_keys_r = merge_keys.copy()
    merge_keys_r.reverse()
    merge_df = merge_df.sort_values(merge_keys_r)

    # Set all types to string
    for key in merge_keys:
        merge_df[key] = merge_df[key].astype(str)

    # Return dataframe
    return merge_df

# Function to get merge dictionary
def get_merge_dictionary(merge_df, merge_keys):
    
    # Consecutive groupby on dataframe
    merge_df_grouped = merge_df
    for i, key in enumerate(merge_keys):
        if i == len(merge_keys)-1:
            break
        # Group by merge keys
        merge_df_grouped = merge_df_grouped.groupby(merge_keys[i+1:]).agg({'index': lambda x: {key: x.tolist()}}).reset_index()
    
    # Create merge dictionary
    merge_dict = {merge_keys[-1]: merge_df_grouped['index'].tolist()}

    # Return merge dictionary
    return merge_dict

# Function to merge datasets
def merge_datasets(ds_ls, merge_dict, max_level=np.inf):
    # Note merge_dict is a dict with a list of dicts len(merge_keys) deep in which the keys are the merge_keys
    # and the values at the deepest level are lists of indices of datasets to be merged

    # Fill merge dict with datasets (recursively)
    def fill_merge_dict_with_datasets(ds_ls, merge_dict):
        for key, val in merge_dict.items():
            if isinstance(val, dict):
                fill_merge_dict_with_datasets(ds_ls, merge_dict[key])
            elif isinstance(val, list):
                for i, val_ in enumerate(val):
                    if isinstance(val_, dict):
                        fill_merge_dict_with_datasets(ds_ls, val_)
                    else:
                        # Replace index with dataset
                        merge_dict[key][i] = ds_ls[val_]
        
        # Return merge dict
        return merge_dict

    # Merge datasets (recursively)
    def merge_datasets(ds_merge_dict):

        # Recursively merge datasets
        def merge_datasets_(ds_merge_dict):
            for key, val in ds_merge_dict.items():
                if isinstance(val, dict):
                    merge_datasets_(ds_merge_dict[key])
                elif isinstance(val, list):
                    if all(isinstance(val_, dict) for val_ in val):
                        for i, val_ in enumerate(val):
                            ds_merge_dict[key][i] = merge_datasets_(val_)
                    
                    # Datasets to be merged
                    elif all(isinstance(val_, xr.Dataset) for val_ in val):
                        if all(key in ds.dims for ds in val):
                            # Concat if key is a dimension
                            ds_merge_dict = xr.concat(val, dim=key)
                        #elif all(key in ds.coords for ds in val):
                        #    # Merge if key is a coordinate
                        #    ds_merge_dict = xr.merge(val)
                        else:                            
                            # Merge if key is a variable
                            ds_merge_dict = xr.merge(val)

            
            # Return merged datasets
            return ds_merge_dict
        
        # Merge datasets until no more dicts in merge_dict
        level = 0
        while isinstance(ds_merge_dict,dict) and level < max_level:
            ds_merge_dict = merge_datasets_(ds_merge_dict)
            level += 1
        
        # Return merged datasets
        return ds_merge_dict
    
    # Fill merge dict with datasets
    ds_merge_dict = fill_merge_dict_with_datasets(ds_ls, deepcopy(merge_dict))

    # Merge datasets
    ds_merged = merge_datasets(deepcopy(ds_merge_dict))
    
    return ds_merged

# Function to count datasets
def count_datasets(ds_cf_merged):
    # Count number of datasets
    def count_datasets_(ds_cf_merged, count):
        if isinstance(ds_cf_merged, xr.Dataset):
            count += 1
        elif isinstance(ds_cf_merged, dict):
            for key, val in ds_cf_merged.items():
                count = count_datasets_(val, count)
        elif isinstance(ds_cf_merged, list):
            for val in ds_cf_merged:
                count = count_datasets_(val, count)
        return count
    
    count = 0 
    return count_datasets_(ds_cf_merged, count)

<IPython.core.display.Javascript object>

In [237]:
merge_keys = ['nvar_stat','ens_perc','rcp','variables']
merge_df = get_merge_dataframe(ds_cf_split_ls, merge_keys)
merge_dict = get_merge_dictionary(merge_df, merge_keys)
ds_cf_merged_dict = merge_datasets(ds_cf_split_ls, merge_dict, max_level=len(merge_keys)-1)

print('Number of datasets: {}'.format(count_datasets(ds_cf_merged_dict)))

Number of datasets: 3


<IPython.core.display.Javascript object>

In [238]:
def merge_ndim(ds_ls, dim):
    # Assign coordinate
    for i in range(len(ds_ls)):
        ds_ls[i] = ds_ls[i].assign_coords({dim: ds_ls[i][dim[1:]].values})

    # Merge datasets
    ds_merged = xr.merge(ds_ls)

    # Drop coordinate
    ds_merged = ds_merged.drop_vars(dim)

    return ds_merged

<IPython.core.display.Javascript object>

In [239]:
ds_cf_merged = merge_ndim(ds_cf_merged_dict['variables'], 'nvar_stat')
print('Number of datasets: {}'.format(count_datasets(ds_cf_merged)))

Number of datasets: 1


<IPython.core.display.Javascript object>

In [240]:
ds_cf_merged

<xarray.Dataset>
Dimensions:   (lat: 181, lon: 361, time: 2, rcp: 2, ens_perc: 5, nvar_stat: 8)
Coordinates:
  * lat       (lat) float32 -90.0 -89.0 -88.0 -87.0 ... 87.0 88.0 89.0 90.0
  * lon       (lon) float32 -180.0 -179.0 -178.0 -177.0 ... 178.0 179.0 180.0
  * time      (time) datetime64[ns] 1991-07-02T12:00:00 2090-07-02T12:00:00
  * rcp       (rcp) float64 4.5 8.5
  * ens_perc  (ens_perc) int32 0 100 5 50 95
    var_stat  (nvar_stat) object b'avg' b'max' b'p10' ... b'p95' b'p99' b'std'
Dimensions without coordinates: nvar_stat
Data variables:
    Dm        (time, lat, lon, rcp, ens_perc, nvar_stat) float32 nan nan ... nan
    Hs        (time, lat, lon, rcp, ens_perc, nvar_stat) float32 nan nan ... nan
    Tm        (time, lat, lon, rcp, ens_perc, nvar_stat) float32 nan nan ... nan

<IPython.core.display.Javascript object>

In [241]:
def make_monotonic(ds):
    for dim in ds.dims:
        ds = ds.sortby(dim)
    return ds

<IPython.core.display.Javascript object>

In [242]:
ds_cf_merged_monotonic = deepcopy(ds_cf_merged)
ds_cf_merged_monotonic = make_monotonic(ds_cf_merged_monotonic)
print('Number of datasets: {}'.format(count_datasets(ds_cf_merged_monotonic)))

Number of datasets: 1


<IPython.core.display.Javascript object>

In [243]:
# Check dataset
ds_cf_merged_monotonic

<xarray.Dataset>
Dimensions:   (lat: 181, lon: 361, time: 2, rcp: 2, ens_perc: 5, nvar_stat: 8)
Coordinates:
  * lat       (lat) float32 -90.0 -89.0 -88.0 -87.0 ... 87.0 88.0 89.0 90.0
  * lon       (lon) float32 -180.0 -179.0 -178.0 -177.0 ... 178.0 179.0 180.0
  * time      (time) datetime64[ns] 1991-07-02T12:00:00 2090-07-02T12:00:00
  * rcp       (rcp) float64 4.5 8.5
  * ens_perc  (ens_perc) int32 0 5 50 95 100
    var_stat  (nvar_stat) object b'avg' b'max' b'p10' ... b'p95' b'p99' b'std'
Dimensions without coordinates: nvar_stat
Data variables:
    Dm        (time, lat, lon, rcp, ens_perc, nvar_stat) float32 nan nan ... nan
    Hs        (time, lat, lon, rcp, ens_perc, nvar_stat) float32 nan nan ... nan
    Tm        (time, lat, lon, rcp, ens_perc, nvar_stat) float32 nan nan ... nan

<IPython.core.display.Javascript object>

In [244]:
# Function to remove indices
def remove_indices(ds):
    # Get dims
    dims = ds.dims

    # Remove unnecessary indices
    for dim in dims:
        try:
            ds = ds.reset_index(dim)
        except:
            continue
    
    # Return dataset
    return ds

ds_cf_merged_monotonic = remove_indices(ds_cf_merged_monotonic)
ds_cf_merged_monotonic

<xarray.Dataset>
Dimensions:   (lat: 181, lon: 361, time: 2, rcp: 2, ens_perc: 5, nvar_stat: 8)
Coordinates:
    lat       (lat) float32 -90.0 -89.0 -88.0 -87.0 ... 87.0 88.0 89.0 90.0
    lon       (lon) float32 -180.0 -179.0 -178.0 -177.0 ... 178.0 179.0 180.0
    time      (time) datetime64[ns] 1991-07-02T12:00:00 2090-07-02T12:00:00
    rcp       (rcp) float64 4.5 8.5
    ens_perc  (ens_perc) int32 0 5 50 95 100
    var_stat  (nvar_stat) object b'avg' b'max' b'p10' ... b'p95' b'p99' b'std'
Dimensions without coordinates: nvar_stat
Data variables:
    Dm        (time, lat, lon, rcp, ens_perc, nvar_stat) float32 nan nan ... nan
    Hs        (time, lat, lon, rcp, ens_perc, nvar_stat) float32 nan nan ... nan
    Tm        (time, lat, lon, rcp, ens_perc, nvar_stat) float32 nan nan ... nan

<IPython.core.display.Javascript object>

In [245]:
import json

# NetCDF attribute alterations by means of metadata template
f_global = open(dir_path_data.joinpath("metadata_waves_by_cowclip.json"))
meta_global = json.load(f_global)


for attr_name, attr_val in meta_global.items():
    if attr_name == 'PROVIDERS':
        attr_val = json.dumps(attr_val)
    ds_cf_merged_monotonic.attrs[attr_name] = attr_val

ds_cf_merged_monotonic.attrs['Conventions'] = "CF-1.8"
ds_cf_merged_monotonic

<xarray.Dataset>
Dimensions:   (lat: 181, lon: 361, time: 2, rcp: 2, ens_perc: 5, nvar_stat: 8)
Coordinates:
    lat       (lat) float32 -90.0 -89.0 -88.0 -87.0 ... 87.0 88.0 89.0 90.0
    lon       (lon) float32 -180.0 -179.0 -178.0 -177.0 ... 178.0 179.0 180.0
    time      (time) datetime64[ns] 1991-07-02T12:00:00 2090-07-02T12:00:00
    rcp       (rcp) float64 4.5 8.5
    ens_perc  (ens_perc) int32 0 5 50 95 100
    var_stat  (nvar_stat) object b'avg' b'max' b'p10' ... b'p95' b'p99' b'std'
Dimensions without coordinates: nvar_stat
Data variables:
    Dm        (time, lat, lon, rcp, ens_perc, nvar_stat) float32 nan nan ... nan
    Hs        (time, lat, lon, rcp, ens_perc, nvar_stat) float32 nan nan ... nan
    Tm        (time, lat, lon, rcp, ens_perc, nvar_stat) float32 nan nan ... nan
Attributes: (12/21)
    TITLE:               Waves by the Coordinated Ocean Wave Climate Project
    TITLE_ABBREVIATION:  waves_by_cowclip
    DESCRIPTION:         Wind-wave conditions (height, period and direction) ...
    SHORT_DESCRIPTION:   Wind-wave conditions as statitics and percentiles fo...
    INSTITUTION:         Coordinated Ocean Wave Climate Project
    PROVIDERS:           {"name": "Coordinated Ocean Wave Climate Project", "...
    ...                  ...
    DOI:                 https://doi.org/10.1038/s41597-020-0446-2
    LONG_NAME:           waves_by_cowclip
    UNITS:               
    COMMENT:             
    CRS:                 EPSG:4326
    Conventions:         CF-1.8

<IPython.core.display.Javascript object>

In [246]:
ds_cf_merged_monotonic.lon.values

array([-180., -179., -178., -177., -176., -175., -174., -173., -172.,
       -171., -170., -169., -168., -167., -166., -165., -164., -163.,
       -162., -161., -160., -159., -158., -157., -156., -155., -154.,
       -153., -152., -151., -150., -149., -148., -147., -146., -145.,
       -144., -143., -142., -141., -140., -139., -138., -137., -136.,
       -135., -134., -133., -132., -131., -130., -129., -128., -127.,
       -126., -125., -124., -123., -122., -121., -120., -119., -118.,
       -117., -116., -115., -114., -113., -112., -111., -110., -109.,
       -108., -107., -106., -105., -104., -103., -102., -101., -100.,
        -99.,  -98.,  -97.,  -96.,  -95.,  -94.,  -93.,  -92.,  -91.,
        -90.,  -89.,  -88.,  -87.,  -86.,  -85.,  -84.,  -83.,  -82.,
        -81.,  -80.,  -79.,  -78.,  -77.,  -76.,  -75.,  -74.,  -73.,
        -72.,  -71.,  -70.,  -69.,  -68.,  -67.,  -66.,  -65.,  -64.,
        -63.,  -62.,  -61.,  -60.,  -59.,  -58.,  -57.,  -56.,  -55.,
        -54.,  -53.,

<IPython.core.display.Javascript object>

In [247]:
# write to NetCDF file to check compliancy

# prevent file locking, see: https://github.com/pydata/xarray/issues/2376
import os
os.environ['HDF5_USE_FILE_LOCKING'] = 'FALSE'

ds_cf_merged_monotonic['time'].attrs.pop('units', None)
ds_cf_merged_monotonic.to_netcdf(path=file_path_cf_merged_nc)

<IPython.core.display.Javascript object>

### Check CF compliancy altered NetCDF files

In [248]:
%%capture cap --no-stderr
# check altered CF compliancy

check_compliancy(testfile=pathlib.Path(file_path_cf_merged_nc), working_dir=dir_path_cf)

<IPython.core.display.Javascript object>

In [249]:
# save altered CF compliancy
save_compliancy(cap, testfile=file_path_cf_merged_nc, working_dir=dir_path_cf)

{'P:\\11209199-climate-resilient-ports\\00_general\\02_COWCLIP_Harvest_Morim_et_al\\output_NetCDF\\waves_by_cowclip_cf_merged.nc': {'warnings': '0', 'errors': '0'}}


<IPython.core.display.Javascript object>

### Write data to Zarr files

In [250]:
# export to Zarr in one-liner (as rp is the temporal dimension)

# export to zarr in write mode (to overwrite if exists)
ds_cf_merged.to_zarr(str(file_path_cf_merged_nc).replace(".nc", ".zarr"), mode="w")

<IPython.core.display.Javascript object>

In [251]:
ds_cf_merged_monotonic

<xarray.Dataset>
Dimensions:   (lat: 181, lon: 361, time: 2, rcp: 2, ens_perc: 5, nvar_stat: 8)
Coordinates:
    lat       (lat) float32 -90.0 -89.0 -88.0 -87.0 ... 87.0 88.0 89.0 90.0
    lon       (lon) float32 -180.0 -179.0 -178.0 -177.0 ... 178.0 179.0 180.0
    time      (time) datetime64[ns] 1991-07-02T12:00:00 2090-07-02T12:00:00
    rcp       (rcp) float64 4.5 8.5
    ens_perc  (ens_perc) int32 0 5 50 95 100
    var_stat  (nvar_stat) object b'avg' b'max' b'p10' ... b'p95' b'p99' b'std'
Dimensions without coordinates: nvar_stat
Data variables:
    Dm        (time, lat, lon, rcp, ens_perc, nvar_stat) float32 nan nan ... nan
    Hs        (time, lat, lon, rcp, ens_perc, nvar_stat) float32 nan nan ... nan
    Tm        (time, lat, lon, rcp, ens_perc, nvar_stat) float32 nan nan ... nan
Attributes: (12/21)
    TITLE:               Waves by the Coordinated Ocean Wave Climate Project
    TITLE_ABBREVIATION:  waves_by_cowclip
    DESCRIPTION:         Wind-wave conditions (height, period and direction) ...
    SHORT_DESCRIPTION:   Wind-wave conditions as statitics and percentiles fo...
    INSTITUTION:         Coordinated Ocean Wave Climate Project
    PROVIDERS:           {"name": "Coordinated Ocean Wave Climate Project", "...
    ...                  ...
    DOI:                 https://doi.org/10.1038/s41597-020-0446-2
    LONG_NAME:           waves_by_cowclip
    UNITS:               
    COMMENT:             
    CRS:                 EPSG:4326
    Conventions:         CF-1.8

<IPython.core.display.Javascript object>

### Write data to CoG (CF compliant)

#### single CoG test

In [294]:
# hard-coded input params
TIME = 0 # select timestep (indices)
RCP = 4.5 # select scenario (indices)
ENS_PERC = 0 # select ensemble
NVAR_STAT = 0 # select variable statistic
VARIABLE = "Dm" # select variable

# open the dataset
ds = xr.open_dataset(file_path_cf_merged_nc)

# make array 2d and fix spatial dimensions and crs
#rds = ds.sel({"ensemble": ENSEMBLE, "nscenarios": SSP}).isel(time=TIME)[VARIABLE]
rds = ds.sel({"rcp": RCP, "ens_perc": ENS_PERC})
rds = rds.isel(time=TIME, nvar_stat=NVAR_STAT)
rds = rds[VARIABLE]
rds = rds.drop_vars(["rcp", "ens_perc", 'time', 'var_stat'])

# set dimensions
rds.rio.set_spatial_dims(x_dim="lon", y_dim="lat")
if not rds.rio.crs:
    rds = rds.rio.write_crs("EPSG:4326")

# convert to dataset
rdsd = rds.to_dataset()

# add all attributes (again)
for attr_name, attr_val in meta_global.items():
    if attr_name == 'PROVIDERS':
        attr_val = json.dumps(attr_val)
    if attr_name == "MEDIA_TYPE": # change media type to tiff, leave the rest as is
        attr_val = "IMAGE/TIFF"
    rdsd.attrs[attr_name] = attr_val

rdsd.attrs['Conventions'] = "CF-1.8"

# Export to COG
rdsd.rio.to_raster(dir_path_cog.joinpath(f"{VARIABLE}.tif"), driver="GTiff")
# export file
#ssp_str = rdsd["scenarios"].item().decode("utf-8") # fix scenario string 
#fname = f"{VARIABLE}_{ssp_str}_ens{float(ENSEMBLE)}_time{TIME}_CF.GeoTiff"
#outpath = cog_dir.joinpath(fname)
#rdsd.rio.to_raster(outpath, driver="GTiff")

<IPython.core.display.Javascript object>

In [298]:
rdsd

<xarray.Dataset>
Dimensions:      (lat: 181, lon: 361)
Coordinates:
  * lat          (lat) float32 -90.0 -89.0 -88.0 -87.0 ... 87.0 88.0 89.0 90.0
  * lon          (lon) float32 -180.0 -179.0 -178.0 -177.0 ... 178.0 179.0 180.0
    spatial_ref  int32 0
Data variables:
    Dm           (lat, lon) float32 nan nan nan nan nan ... nan nan nan nan nan
Attributes: (12/21)
    TITLE:               Waves by the Coordinated Ocean Wave Climate Project
    TITLE_ABBREVIATION:  waves_by_cowclip
    DESCRIPTION:         Wind-wave conditions (height, period and direction) ...
    SHORT_DESCRIPTION:   Wind-wave conditions as statitics and percentiles fo...
    INSTITUTION:         Coordinated Ocean Wave Climate Project
    PROVIDERS:           {"name": "Coordinated Ocean Wave Climate Project", "...
    ...                  ...
    DOI:                 https://doi.org/10.1038/s41597-020-0446-2
    LONG_NAME:           waves_by_cowclip
    UNITS:               
    COMMENT:             
    CRS:                 EPSG:4326
    Conventions:         CF-1.8

<IPython.core.display.Javascript object>

In [299]:
ds

<xarray.Dataset>
Dimensions:   (lat: 181, lon: 361, time: 2, rcp: 2, ens_perc: 5, nvar_stat: 8)
Coordinates:
  * lat       (lat) float32 -90.0 -89.0 -88.0 -87.0 ... 87.0 88.0 89.0 90.0
  * lon       (lon) float32 -180.0 -179.0 -178.0 -177.0 ... 178.0 179.0 180.0
  * time      (time) datetime64[ns] 1991-07-02T12:00:00 2090-07-02T12:00:00
  * rcp       (rcp) float64 4.5 8.5
  * ens_perc  (ens_perc) int32 0 5 50 95 100
    var_stat  (nvar_stat) |S3 ...
Dimensions without coordinates: nvar_stat
Data variables:
    Dm        (time, lat, lon, rcp, ens_perc, nvar_stat) float32 ...
    Hs        (time, lat, lon, rcp, ens_perc, nvar_stat) float32 ...
    Tm        (time, lat, lon, rcp, ens_perc, nvar_stat) float32 ...
Attributes: (12/21)
    TITLE:               Waves by the Coordinated Ocean Wave Climate Project
    TITLE_ABBREVIATION:  waves_by_cowclip
    DESCRIPTION:         Wind-wave conditions (height, period and direction) ...
    SHORT_DESCRIPTION:   Wind-wave conditions as statitics and percentiles fo...
    INSTITUTION:         Coordinated Ocean Wave Climate Project
    PROVIDERS:           {"name": "Coordinated Ocean Wave Climate Project", "...
    ...                  ...
    DOI:                 https://doi.org/10.1038/s41597-020-0446-2
    LONG_NAME:           waves_by_cowclip
    UNITS:               
    COMMENT:             
    CRS:                 EPSG:4326
    Conventions:         CF-1.8

<IPython.core.display.Javascript object>

#### Multiple CoGs

In [348]:
# do for all CoGs (CF compliant)

# open the dataset
ds = xr.open_dataset(file_path_cf_merged_nc)

# extract list of data variables
for i, time in enumerate(ds.time.values):
    for j, rcp in enumerate(ds.rcp.values):
        for k, ens_perc in enumerate(ds.ens_perc.values):
            for l, nvar_stat in enumerate(ds.nvar_stat.values):
                var_stat = ds.var_stat.values[l].decode("utf-8")
                for m, var in enumerate(list(set(ds.variables) - set(ds.dims) - set(ds.coords))):
                    # make array 2d and fix spatial dimensions and crs
                    ds_ = ds.copy()
                    ds_ = ds_.isel({"time": i, "rcp": j, "ens_perc": k, "nvar_stat": l})
                    ds_ = ds_[var]
                    ds_ = ds_.drop_vars(["rcp", "ens_perc", 'time', 'var_stat'])

                    # Skip if all data is nan
                    if np.isnan(ds_.values).all():
                        print('skipped')

                    # Set dimensions
                    ds_.rio.set_spatial_dims(x_dim="lon", y_dim="lat")
                    if not ds_.rio.crs:
                        ds_ = ds_.rio.write_crs("EPSG:4326")

                    # Get file path
                    file_path_COG = os.path.join(dir_path_cog, f'time={str(time)[:4]}', f'rcp={rcp}', f'ens_perc={ens_perc}', f'var_stat={var_stat}', f'{var}.tif')
                    if not os.path.exists(os.path.dirname(file_path_COG)):
                        os.makedirs(os.path.dirname(file_path_COG))
                    
                    # Set overwrite is false because tifs should be unique
                    try:
                        write_cog(ds_, file_path_COG, overwrite=False)
                    except OSError as e:
                        continue
                    

skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped


<IPython.core.display.Javascript object>